# Homework Starter — Stage 04: Data Acquisition and Ingestion
Name: Mingjia Jin
Date: 

## Objectives
- API ingestion with secrets in `.env`
- Scrape a permitted public table
- Validate and save raw data to `data/raw/`

In [1]:
import os
from datetime import datetime
import pathlib
from pathlib import Path
import requests # a Python library that allows you to make HTTP requests — like visiting a website, but with code.
import pandas as pd
from bs4 import BeautifulSoup # Scrape Data
from dotenv import load_dotenv
import numpy as np

In [2]:
RAW = pathlib.Path('data/raw')
RAW.mkdir(parents=True, exist_ok=True)

load_dotenv()
print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY'))) # yfinance do not need this one

ALPHAVANTAGE_API_KEY loaded? False


## Helpers (use or modify)

In [3]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S') # return current system time like 20250812-095532

In [4]:
PROJECT_ROOT = Path.cwd().parent # Current Working Directory
DATA_RAW = PROJECT_ROOT / "data" / "raw" 

def save_csv(df: pd.DataFrame, prefix: str, source: str, ticker: str, ext: str = "csv") -> Path:
    # Generate timestamp like '20250824-1052'
    ts = datetime.now().strftime("%Y%m%d-%H%M")
    # Ensure the folder exists
    DATA_RAW.mkdir(parents=True, exist_ok=True)
    # Construct the file path
    path = DATA_RAW / f"{prefix}_{source}_{ticker}_{ts}.{ext}"
    # Save the DataFrame
    df.to_csv(path, index=True)  # index=True because Date is index
    print("Saved:", path)

    return path

Validate function requirements:

| Requirement          | Meaning                                                  | Example                                                                                              |
| -------------------- | -------------------------------------------------------- | ---------------------------------------------------------------------------------------------------- |
| **Required columns** | The DataFrame must contain certain columns               | Stock data must include `Date, Open, High, Low, Close, Volume`                                       |
| **NA counts**        | The number of missing values (NA/NaN) must be reasonable | For example, the `Volume` column may allow up to 20% missing values, but it cannot be entirely empty |
| **Shape**            | The dataset must have enough rows, not just a few        | At least 100 rows of data are required; otherwise, the dataset is considered invalid                 |



In [5]:
def validate(df: pd.DataFrame, required, expected_dtypes):
    missing = [c for c in required if c not in df.columns]
    
    mismatch = {} # data type mismatch
    for col, expected_type in expected_dtypes.items():
        if col in df.columns:
            actual = df[col].dtype
            if not np.issubdtype(actual, expected_type):
                mismatch[col] = str(actual)
                
    return {
        'missing': missing,
        'shape': df.shape,
        'na_total': int(df.isna().sum().sum()),
        'mismatched data type': mismatch,
        'index is': df.index.name
    }

## Part 1 — API Pull (Required)
`yfinance` here

Choose an endpoint (e.g., Alpha Vantage or use `yfinance` fallback).

In [6]:
ticker = 'AAPL'
START = "2015-01-01"
END   = None

USE_ALPHA = bool(os.getenv('ALPHAVANTAGE_API_KEY'))
if USE_ALPHA:
    url = 'https://www.alphavantage.co/query'
    params = {'function':'TIME_SERIES_DAILY_ADJUSTED','symbol':SYMBOL,'outputsize':'compact','apikey':os.getenv('ALPHAVANTAGE_API_KEY')}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    key = [k for k in js if 'Time Series' in k][0]
    df_api = pd.DataFrame(js[key]).T.reset_index().rename(columns={'index':'date','5. adjusted close':'adj_close'})[['date','adj_close']]
    df_api['date'] = pd.to_datetime(df_api['date']); df_api['adj_close'] = pd.to_numeric(df_api['adj_close'])
else:
    import yfinance as yf
    df_api = yf.Ticker(ticker).history(start=START, end=END, auto_adjust=False).drop(columns=["Dividends", "Stock Splits"])
    # df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()[['Date','Open','High','Low','Close','Adj Close','Volume']]

print(df_api)

                                 Open        High         Low       Close  \
Date                                                                        
2015-01-02 00:00:00-05:00   27.847500   27.860001   26.837500   27.332500   
2015-01-05 00:00:00-05:00   27.072500   27.162500   26.352501   26.562500   
2015-01-06 00:00:00-05:00   26.635000   26.857500   26.157499   26.565001   
2015-01-07 00:00:00-05:00   26.799999   27.049999   26.674999   26.937500   
2015-01-08 00:00:00-05:00   27.307501   28.037500   27.174999   27.972500   
...                               ...         ...         ...         ...   
2025-08-18 00:00:00-04:00  231.699997  233.119995  230.110001  230.889999   
2025-08-19 00:00:00-04:00  231.279999  232.869995  229.350006  230.559998   
2025-08-20 00:00:00-04:00  229.979996  230.470001  225.770004  226.009995   
2025-08-21 00:00:00-04:00  226.270004  226.520004  223.779999  224.899994   
2025-08-22 00:00:00-04:00  226.169998  229.089996  225.410004  227.759995   

In [7]:
required_cols = ["Date", "Open", "High", "Low", "Close", 'Adj Close', "Volume"]
expected_types = {
    "Date": np.datetime64,
    "Open": np.number,
    "High": np.number,
    "Low": np.number,
    "Close": np.number,
    "Adj Close": np.number,
    "Volume": np.number
}

v_api = validate(df_api, required_cols, expected_types)
print(v_api)

{'missing': ['Date'], 'shape': (2676, 6), 'na_total': 0, 'mismatched data type': {}, 'index is': 'Date'}


In [8]:
# save data
save_csv(df_api, prefix="api", source="yfinance", ticker="AAPL")

Saved: /Users/fd/gitlocal/bootcamp_mingjia_jin/project/data/raw/api_yfinance_AAPL_20250824-1545.csv


PosixPath('/Users/fd/gitlocal/bootcamp_mingjia_jin/project/data/raw/api_yfinance_AAPL_20250824-1545.csv')

## Part 2 — Scrape a Public Table (Required)
Replace `SCRAPE_URL` with a permitted page containing a simple table.

In [9]:
SCRAPE_URL = 'https://example.com/markets-table'  # TODO: replace with permitted page
headers = {'User-Agent':'AFE-Homework/1.0'}

try:
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30); resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)

# This fallback is not returning useful data — it’s just there to prevent the notebook from crashing.
# So, ignore this.
######################################################################################################
# except Exception as e:
#     print('Scrape failed, using inline demo table:', e)
#     html = '<table><tr><th>Ticker</th><th>Price</th></tr><tr><td>AAA</td><td>101.2</td></tr></table>'
#     soup = BeautifulSoup(html, 'html.parser')
#     rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
#     header, *data = [r for r in rows if r]
#     df_scrape = pd.DataFrame(data, columns=header)

# if 'Price' in df_scrape.columns:
#     df_scrape['Price'] = pd.to_numeric(df_scrape['Price'], errors='coerce')
# v_scrape = validate(df_scrape, list(df_scrape.columns)); v_scrape

SyntaxError: incomplete input (3631983383.py, line 24)

In [22]:
# --- Target
SCRAPE_URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
headers = {"User-Agent": "AFE-Homework/1.0"}

# --- Get and parse
resp = requests.get(SCRAPE_URL, headers=headers, timeout=30)
resp.raise_for_status() # It checks the HTTP response status code and raises an error if the request failed(404 or 500). 
soup = BeautifulSoup(resp.text, "html.parser") # resp.text： orginal web code; "html.parser": decoder, Python’s built-in HTML parser.

# --- Find table
table = soup.find("table", {"id": "constituents"})

# --- Extract rows
rows = []
for tr in table.find_all("tr"):
    cols = [td.get_text(strip=True) for td in tr.find_all(["td", "th"])]
    if cols:
        rows.append(cols)

# --- Create DataFrame
header = rows[0]
data = rows[1:]
df_scrape = pd.DataFrame(data, columns=header)

In [27]:
# --- Target
SCRAPE_URL = "https://www.worldgovernmentbonds.com/country/united-states/"
headers = {"User-Agent": "AFE-Homework/1.0"}

# --- Get and parse
resp = requests.get(SCRAPE_URL, headers=headers, timeout=30)
resp.raise_for_status()  # Raise error for 404, 500 etc
soup = BeautifulSoup(resp.text, "html.parser")  # Convert HTML string into DOM tree

# --- Find table
# The table with U.S. bond yields has class 'table' and is the first one that matches bond layout
# table = soup.find("table", class_="table")  # Or use soup.find_all("table")[0] if unsure
table = soup.find_all("table")[0] 

# --- Extract rows
rows = []
for tr in table.find_all("tr"):
    cols = [td.get_text(strip=True) for td in tr.find_all(["td", "th"])] # Inside each row, finds all <td> and <th> cells.
    if cols:
        rows.append(cols)

# --- Create DataFrame
header = rows[0]
data = rows[1:]
df_scrape = pd.DataFrame(data, columns=header)

df_scrape

,,Interest Rates,Value
0,,Central Bank Rate,----%


In [10]:
from bs4 import BeautifulSoup
import requests

# Setup
headers = {'User-Agent': 'Mozilla/5.0 ... Safari/601.3.9'}
url = 'https://finance.yahoo.com/most-active'

# Fetch the page
response = requests.get(url, headers=headers)

# Parse HTML using BeautifulSoup
soup = BeautifulSoup(response.content, 'lxml')

# Each row of data is within elements having class 'simpTblRow'
for item in soup.select('.simpTblRow'):
    print(item)
    print('------------------')


In [12]:
# --- Target
URL = "https://www.federalreserve.gov/releases/h15/data.htm"
headers = {"User-Agent": "AFE-Homework/1.0"}

# --- Get and parse
resp = requests.get(URL, headers=headers)
resp.raise_for_status()
soup = BeautifulSoup(resp.content, "html.parser")

# --- Find all tables (each rate has one table)
tables = soup.find_all("table")

# Let's take the first table (usually 1-Year Treasury Yield or similar)
table = tables[0]

# --- Extract rows
rows = []
for tr in table.find_all("tr"):
    cols = [td.get_text(strip=True) for td in tr.find_all(["td", "th"])]
    if cols:
        rows.append(cols)

print(rows)
# --- Create DataFrame
# header = rows[0]
# data = rows[1:]
# df = pd.DataFrame(data, columns=header)
# df

IndexError: list index out of range

In [ ]:
# --- Optional: save to CSV
Path("data/raw").mkdir(parents=True, exist_ok=True)
ts = datetime.now().strftime("%Y%m%d-%H%M")
csv_path = f"data/raw/scrape_us_bond_yields_{ts}.csv"
df_scrape.to_csv(csv_path, index=False)
print("✅ Saved:", csv_path)

# --- Preview
df_scrape.head()

In [ ]:
# --- Save
Path("data/raw").mkdir(parents=True, exist_ok=True)
ts = datetime.now().strftime("%Y%m%d-%H%M")
df.to_csv(f"data/raw/scrape_wiki_sp500_{ts}.csv", index=False)
print("✅ Saved:", f"data/raw/scrape_wiki_sp500_{ts}.csv")

In [ ]:
# save raw data
save_csv(df_api, prefix="api", source="yahoo", ident="AAPL")

save_csv(df_scrape, prefix='scrape', site='example', table='markets')